In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

VECTOR_DIR = "vector_store"

# Inicializar embeddings
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

# Cargar base vectorial
db = Chroma(
    persist_directory=VECTOR_DIR,
    embedding_function=embeddings
)

query = "¿Qué condiciones particulares tiene el contrato?"
resultados = db.similarity_search(query, k=3)

for r in resultados:
    print(f"Documento: {r.metadata['source']}")
    print(r.page_content)
    print("-" * 50)


C:\Users\Odaliz Aracely\AppData\Local\Temp\ipykernel_8428\1235951705.py:17: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(


Documento: condiciones_particulares_del_pliego_chunks.txt
condiciones adicionales que considere pertinentes.)  
5.3  Obligaciones de la contratante: 
 
- Designar al administrador del contrato. 
 
- Dar solución a las peticiones y problemas que se presentaren en la ejecución del 
contrato, en un plazo (número de días) co ntados a partir de la petición escrita 
formulada por el contratista. 
 
- Para el caso de servicios, de ser necesari o, previo el trámite legal y administrativo 
respectivo, celebrar los contratos complementarios en un plazo ( número de días ) 
contados a partir de la decisión de la máxima autoridad. 
 
- Suscribir las actas de entrega recepción correspondientes, siempre que se haya 
cumplido con lo previsto en la ley para  la entrega recepción; y, en general, 
cumplir con las obligaciones derivadas del contrato. 
 (Acorde con la naturaleza de la contratación la entidad contratante podrá establecer las condiciones adicionales que considere pertinentes.)
--------------

In [4]:
import os
import json
from dotenv import load_dotenv
load_dotenv()

from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI

VECTOR_DIR = "vector_store"

# Inicializar embeddings
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

# Cargar base vectorial
db = Chroma(
    persist_directory=VECTOR_DIR,
    embedding_function=embeddings
)

# Consulta para obtener documentos similares
query = "¿Qué condiciones particulares tiene el contrato?"
resultados = db.similarity_search(query, k=3)

# Concatenar textos para enviar al modelo
texto_combinado = "\n".join([r.page_content for r in resultados])

# Prompt para pedir JSON estricto
prompt_template = """
Eres un asistente que extrae información relevante de un texto legal para procesos de licitación. 
Devuelve solo un JSON válido (sin texto adicional ni explicaciones), con esta estructura:

{{
  "clausulas_legales": "...",
  "requisitos_tecnicos": "...",
  "condiciones_economicas": "..."
}}

Texto:
{texto}
"""

prompt = PromptTemplate(
    input_variables=["texto"],
    template=prompt_template
)

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
chain = LLMChain(llm=llm, prompt=prompt)

# Ejecutar el chain
respuesta = chain.run(texto=texto_combinado)

# Intentar convertir la respuesta a JSON, con manejo de error
try:
    resultados_finales = json.loads(respuesta)
except json.JSONDecodeError:
    print("⚠️ Error al parsear JSON. Respuesta recibida:")
    print(respuesta)
    resultados_finales = {}

# Mostrar resultado final (opcional)
print(resultados_finales)


C:\Users\Odaliz Aracely\AppData\Local\Temp\ipykernel_8428\1469207394.py:53: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)


{'clausulas_legales': 'Obligaciones de la contratante: Designar al administrador del contrato, dar solución a las peticiones y problemas en un plazo determinado, celebrar contratos complementarios en un plazo determinado, suscribir actas de entrega recepción, cumplir con las obligaciones derivadas del contrato. Aceptación voluntaria de las partes a lo convenido en el contrato.', 'requisitos_tecnicos': 'El plazo para la entrega de bienes o prestación de servicios es de un número de días, contado a partir de la fecha de entrega del anticipo o suscripción del contrato.', 'condiciones_economicas': 'Por cada día de retardo en la ejecución de las obligaciones contractuales por parte del contratista, se aplicará una multa de un valor establecido por la entidad contratante.'}


In [6]:
import os
import json
from dotenv import load_dotenv
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

load_dotenv()

VECTOR_DIR = "vector_store"
OUTPUT_JSON = "processed/extracted_info.json"

# Inicializar embeddings y vector DB
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

db = Chroma(
    persist_directory=VECTOR_DIR,
    embedding_function=embeddings
)

# Inicializar LLM para extracción
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# Prompt template para extraer datos estructurados en JSON
prompt_template = """
Extrae la siguiente información del siguiente texto del contrato en formato JSON:
{secciones}

Texto:
{texto}

Devuelve sólo el JSON.
"""

def extraer_info(texto, secciones):
    prompt = PromptTemplate(
        input_variables=["texto", "secciones"],
        template=prompt_template
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    respuesta = chain.run(texto=texto, secciones=secciones)

    # Limpiar la respuesta para quitar ```json y ``` si existen
    respuesta_limpia = respuesta.strip()
    if respuesta_limpia.startswith("```json"):
        respuesta_limpia = respuesta_limpia[len("```json"):].strip()
    if respuesta_limpia.endswith("```"):
        respuesta_limpia = respuesta_limpia[:-3].strip()

    try:
        resultado_json = json.loads(respuesta_limpia)
    except json.JSONDecodeError:
        # En caso de que la respuesta no sea un JSON válido, devolvemos un error
        resultado_json = {
            "error": "No se pudo parsear la respuesta como JSON",
            "respuesta": respuesta,
            "respuesta_limpia": respuesta_limpia
        }
    return resultado_json

def consulta_y_extraccion(query, secciones):
    resultados = db.similarity_search(query, k=5)
    texto_combinado = "\n".join([r.page_content for r in resultados])
    info = extraer_info(texto_combinado, secciones)
    return info

def guardar_json(data, path):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

# Ejemplo uso con varias secciones a extraer
if __name__ == "__main__":
    secciones_a_extraer = """
    {
      "clausulas_legales": "Obligaciones, responsabilidades y aceptación de las partes",
      "requisitos_tecnicos": "Plazos, entregables, especificaciones técnicas",
      "condiciones_economicas": "Multas, pagos, presupuestos y garantías"
    }
    """

    consulta = "condiciones del contrato"

    info_extraida = consulta_y_extraccion(consulta, secciones_a_extraer)

    print("=== Información extraída ===")
    print(json.dumps(info_extraida, indent=4, ensure_ascii=False))

    guardar_json(info_extraida, OUTPUT_JSON)
    print(f"✅ Información guardada en {OUTPUT_JSON}")


=== Información extraída ===
{
    "clausulas_legales": "Obligaciones, responsabilidades y aceptación de las partes",
    "requisitos_tecnicos": "Plazos, entregables, especificaciones técnicas",
    "condiciones_economicas": "Multas, pagos, presupuestos y garantías"
}
✅ Información guardada en processed/extracted_info.json
